# Demo with Dark Frame Correction and TIFF Export

This demo uses an IOC called `mini_beamline` that simulates a beamline, complete with a image detector that has a nonzero "dark image" when the shutter is closed. The IOC is running in the background. We can check on it --- this should say `RUNNING`. There are some other components here as well which we aren't using for this demo but may be useful for future demos.

In [ ]:
!supervisorctl -c supervisor/supervisord.conf status

The startup script, akin to the IPython profile used at the beamline, defines ophyd Devices that connect to their counterparts on the IOC. It defines a RunEngine `RE` with several customizations.

In [ ]:
%run startup.py

At the start of every Run (i.e. ``scan`` or ``count``) it will insert a copy of the most recent dark frame. It will decide to take a new dark frame if the most recent dark frame is oldest than this configurable setting (units: seconds).

In [ ]:
take_dark_frames.max_age

Documents generated by `RE` are sent to a series of functions that:

* Access and read the files generated by the area detector (light or dark frames).
* Perform dark-subtraction.
* Export the dark-subtracted files to disk. This occurs in a streaming fashion during the scan, so images are immedaitely available. The sample name is included in the filename.

The `startup.py` file also defines some custom plans, light variations on built-in bluesky plans. They assume that samples are arranged in a line at various position. This ``samples`` dict maps each sample's position along that line to a dict of metadata about it (which can contain anything).

In [ ]:
samples = {1: {'composition': 'Ni', 'name': 'S1'},                                                                                                                                          
           7: {'composition': 'LaB6', 'name': 'S1E'}}                                                                                                                                       

Perform a `count` on each sample in turn. Each sample's data will be saved in a different Run, and files will be generated in the irectory ``exported/``. (That export path is hard-coded in `startup.py`, but it's straightforward to make it configurable.)

In [ ]:
RE(multi_sample_count(samples))    
# This issues a warning about duplicates that you can safely ignore...will fix later.

In [ ]:
!ls exported

Now perform a `rel_grid_scan` in the region of each sample.

In [ ]:
help(multi_sample_grid)

In [ ]:
RE(multi_sample_grid(samples, 1, 1, 3, 3))

In [ ]:
!ls exported/